## other idea
remove duplicates using running time 
    -> problem because same movies have 1 minute difference in running time in the two data sets
try to use anoter dataset than basics that has things in common with our dataset for removal of duplicates

### Ideal
finding mapping between movie id

data set:
https://datasets.imdbws.com/

In [1]:
import pandas as pd
import numpy as np

In [2]:
movie = pd.read_csv("MovieSummaries/movie.metadata.tsv", delimiter='\t',header=None)
character = pd.read_csv("MovieSummaries/character.metadata.tsv", delimiter='\t',header=None)
movie.columns = ['Wikipedia movie ID','Freebase movie ID','Movie name','Movie release date','Movie box office revenue','Movie runtime','Movie languages','Movie countries','Movie genres']
movie["Movie name"] = movie["Movie name"].str.capitalize()
movie["Movie name"] = movie["Movie name"].str.replace(r'[éè]', 'e', regex=True)

In [4]:
# imdb1 has ratings, imdb identifier and number of votes
imdb1 = pd.read_csv('title.ratings.tsv', delimiter='\t')
# imdb2 has genres, title, year, etc
imdb2 = pd.read_csv('title.basics.tsv', delimiter='\t', low_memory=False)


In [5]:
merged_df = pd.merge(imdb1, imdb2, on='tconst', how='inner')

In [6]:
# in the imdb data set they include series and a lot of other things.
#  This removes a lot of movies that are not in the other data 
merged_2 = merged_df[(merged_df.titleType == 'movie')|(merged_df.titleType == 'tvMovie')]
# removing all the movies that have less than 30 votes 
# questionable step because some of the movies in our data set have 
# very low amount of votes
merged_3= merged_2[merged_2['numVotes'] >= 30]
# making the titles of the movies in the same format for both data set 
# for example in one data set a movie can be called "The matrix" and in the other "The Matrix"
merged_3.loc[:, 'primaryTitle'] = merged_3['primaryTitle'].str.capitalize()

In [7]:
# removing all the columns that are not needed
merged_4 = merged_3.drop(columns=['numVotes' ,'titleType', 'isAdult', 'endYear', 'originalTitle', 'runtimeMinutes', 'genres', 'tconst'])

In [8]:
# merging the data sets on our movie data set
merged_final = pd.merge(merged_4, movie, left_on='primaryTitle', right_on='Movie name', how='right')

In [9]:
# creating a column that has the year of the movie release so that we can remove duplicates 
# that have the same name but different release year
merged_final["Movie release year"] = merged_final["Movie release date"].str[:4]

In [10]:
# trying to remove duplicates
merged_final2 = merged_final[merged_final['Movie release year']==merged_final['startYear']]

In [11]:
# drop colllumn that are not needed
merged_final2 = merged_final2.drop(columns=['primaryTitle', 'Movie release year', 'startYear', 'Movie release date'])

In [26]:
merged_final2 = merged_final2[merged_final2['Movie box office revenue'].notna()]

# The best data right now

In [27]:
merged_final2

,averageRating,Wikipedia movie ID,Freebase movie ID,Movie name,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,4.9,975900,/m/03vyhn,Ghosts of mars,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
9,6.8,10408933,/m/02qc0j7,Alexander's ragtime band,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ..."
17,7.5,171005,/m/016ywb,Henry v,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa..."
21,7.8,77856,/m/0kcn7,Mary poppins,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""..."
35,6.5,156558,/m/014k4y,Baby boy,29381649.0,123.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
...,...,...,...,...,...,...,...,...,...
122335,6.9,1191380,/m/04f_y7,Wilde,2158775.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...","{""/m/0hn10"": ""LGBT"", ""/m/017fp"": ""Biography"", ..."
122336,7.1,54540,/m/0f7hw,Coming to america,288752301.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""..."
122388,5.3,7761830,/m/0kvgqb,Spaced invaders,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3mt0"": ""Alien Film"", ""/m/06n90"": ""Scie..."
122393,6.7,1918494,/m/0660qx,State and main,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana..."


In [20]:
merged_final2.shape[0]-merged_final2["Movie box office revenue"].isnull().sum()

7440

In [19]:
movie.shape[0]-movie["Movie box office revenue"].isnull().sum()

8401

In [23]:
merged_final['startYear'] = pd.to_numeric(merged_final['startYear'], errors='coerce').astype('Int64')

In [25]:
merged_final['startYear'].sort_values(ascending=False)

28863     2024
81302     2024
60430     2024
44753     2024
9767      2024
          ... 
122400    <NA>
122401    <NA>
122409    <NA>
122413    <NA>
122414    <NA>
Name: startYear, Length: 122415, dtype: Int64